### Roland Farrell 05/15/22
### see arXiv:2207.01731

In [4]:
import numpy as np
import qiskit
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt
from qiskit import Aer, transpile
from qiskit.circuit import Parameter, QuantumRegister
import random
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import TransformationPass

In [2]:
from qiskit import IBMQ

# Put in personal login token
token=""
provider = IBMQ.enable_account(token)
IBMQ.active_account()
IBMQ.providers() 
# Put in provider information
provider = IBMQ.get_provider(hub="", group="", project="")

# Choose the desired quantum hardware
backend = provider.get_backend('ibm_perth')

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-research-2', group='uni-washington-1', project='main')>,
 <IBMQSimulat

# Construct the ciruit which implements Trotterized time evolution for one flavor QCD in one dimension 

### Hopping between a given quark and antiquark. Start is index of the quark qubit and t is the time.

In [4]:
def UkinAdd(circ,t,start):
    end = start+3
    
    # Control rotation if 01 or 10
    circ.cx(end,start)
    
    # Rx controlled on 10, 01. start+1 and start+2 control +/-
    circ.h(end)
    circ.cx(start+1,start+2)
    circ.cx(start+2,start+3)
    circ.rz(t/2,end)
    circ.cx(start,end)
    circ.rz(-t/2,end)
    circ.cx(start,end)
    circ.cx(start+2,start+3)
    circ.cx(start+1,start+2)
    circ.h(end)
    circ.cx(end,start)

### Adds a ZZ rotation, indices is a tuple labelling the two qubits. ph is the phase of the rotation.

In [5]:
def Uel1Add(circ,indices,ph):
    circ.cx(indices[0],indices[1])
    circ.rz(ph,indices[1])
    circ.cx(indices[0],indices[1])

### Trotterization of the mass part of the Hamiltonian. m is the mass and t is the time.

In [6]:
def Um(circ, m, t):
    for n in range(2):
        # For each color
        for c in range(3):
            circ.rz((-1) ** (n) * m * t, 3*n + c)

### Trotterization of the kinetic part of the Hamiltonian. t is the time.

In [7]:
def Ukin(circ, t):
    # Add each term in sum to the circuit
    for q0 in range(3):
        UkinAdd(circ, t, q0)

### Trotterization of the chromo-electric part of the Hamiltonian. g is the gauge coupling and t is the time.

In [8]:
def Uel1(circ,g,t):
    Uel1Add(circ,(0, 1),(-t * g ** 2/3))
    Uel1Add(circ,(0, 2),(-t * g ** 2/3))
    Uel1Add(circ,(1, 2),(-t * g ** 2/3))

# A class which twirls the CNOTs in a given circuit. This replaces a CNOT with an equivalent circuit element in order to reduce the effects of correlated noise; see arXiv:2205.09247.

In [9]:
class CXTranslator(TransformationPass):
    # A transpiler pass to replace CX gates with twirled CX gates.

    def run(self, dag):

        # iterate over all operations
        for node in dag.op_nodes():

            # if a CNOT then replace with equivalent ``twirled" circuit element. 
            if node.op.name == "cx":
                rep = QuantumCircuit(2)
                rand = random.randrange(16)
                
                if rand == 0:
                    rep.cx(0,1)
                if rand == 1:
                    rep.x(1)
                    rep.cx(0,1)
                    rep.x(1)
                if rand == 2:
                    rep.y(1)
                    rep.cx(0,1)
                    rep.z(0)
                    rep.y(1)
                if rand == 3:
                    rep.z(1)
                    rep.cx(0,1)
                    rep.z(0)
                    rep.z(1)
                if rand == 4:
                    rep.x(0)
                    rep.cx(0,1)
                    rep.x(0)
                    rep.x(1)
                if rand == 5:
                    rep.x(0)
                    rep.x(1)
                    rep.cx(0,1)
                    rep.x(0)
                if rand == 6:
                    rep.x(0)
                    rep.y(1)
                    rep.cx(0,1)
                    rep.y(0)
                    rep.z(1)
                if rand == 7:
                    rep.x(0)
                    rep.z(1)
                    rep.cx(0,1)
                    rep.y(0)
                    rep.y(1)
                if rand == 8:
                    rep.y(0)
                    rep.cx(0,1)
                    rep.y(0)
                    rep.x(1)
                if rand == 9:
                    rep.y(0)
                    rep.x(1)
                    rep.cx(0,1)
                    rep.y(0)
                if rand == 10:
                    rep.y(0)
                    rep.y(1)
                    rep.cx(0,1)
                    rep.x(0)
                    rep.z(1)
                if rand == 11:
                    rep.y(0)
                    rep.z(1)
                    rep.cx(0,1)
                    rep.x(0)
                    rep.y(1)
                if rand == 12:
                    rep.z(0)
                    rep.cx(0,1)
                    rep.z(0)
                if rand == 13:
                    rep.z(0)
                    rep.x(1)
                    rep.cx(0,1)
                    rep.z(0)
                    rep.x(1)
                if rand == 14:
                    rep.z(0)
                    rep.y(1)
                    rep.cx(0,1)
                    rep.y(1)
                if rand == 15:
                    rep.z(0)
                    rep.z(1)
                    rep.cx(0,1)
                    rep.z(1)
                    
                # replace the CNOT with the new twirled circuit element
                dag.substitute_node_with_dag(node, circuit_to_dag(rep))

        return dag

# Generate the Trotterization circuit, transpiled to run on the specified device architecture.

In [10]:
# The mass, m, and gauge coupling, g
m=1
g=1

# Keep the evolution time as a parameter. 
# This prevents the mitigation circuit with t=0 from being simplified by the transpiler.
t1 = Parameter("t1")

circ=QuantumCircuit(6,6)
# Initialize the trivial vacuum
circ.x(0)
circ.x(1)
circ.x(2)

# One second order Trotter step with terms explicitly combined and making use of [Um, Uel]=0 to cancel CNOTs.
# Last sequence is ommited since it is just ZZ rotations which don't effect measurement.
Um(circ, m, t1, 2)
Uel1(circ, g, t1, 2)
Ukin(circ, t1, 2)

circ.measure([0,1,2,3,4,5],[0,1,2,3,4,5])

# Search for the optimal way of mapping to architecture of the device on the backend
minm = 500
for i in range(50):  
    circ1 = transpile(circ,backend=backend,optimization_level=3)
    if circ1.count_ops()["cx"] < minm:
        minm = circ1.count_ops()["cx"]
        circ2 = circ1
circ = circ2

### Creating the measurement mitigation circuit

In [3]:
qr = QuantumRegister(6)
mit_pattern=[[i] for i in range(6)]
meas_calibs, state_labels = tensored_meas_cal(mit_pattern, qr=qr, circlabel='mcal')
circ_meas = transpile(meas_calibs, backend)
circ_list = circ_meas

NameError: name 'backend' is not defined

### Twirling the Trotterization circuit

In [15]:
# A trotter step of Delta t = 3
t=3
i=0
myruns = 149
while i < myruns:
    # Twirl the physics circuit
    circTrot = circ.bind_parameters([t])
    circTrot = CXTranslator()(circTrot)
    
    # Twirl the mitigation circuit
    circMiti = circ.bind_parameters([0])
    circMiti = CXTranslator()(circMiti)

    # Light transpile
    circTrot = transpile(circTrot, backend=backend, optimization_level=1, translation_method="translator", routing_method="none", layout_method = "trivial")
    circMiti = transpile(circMiti, backend=backend, optimization_level=1,translation_method="translator", routing_method="none", layout_method = "trivial")
    if circMiti.count_ops()["cx"] == ncx:
        circ_list.append(circTrot)
        circ_list.append(circMiti)
        i+=1

# Running the circuit on the quantum device

In [25]:
nshots = 1000
job = backend.run(circ_list, shots=nshots)
results = job.result()

meas_fitter = TensoredMeasFitter(results, mit_pattern, circlabel='mcal')
meas_filter = meas_fitter.filter
# Apply measurement mitigation to the results
mitigated_results = meas_filter.apply(results)
mit_output=mitigated_results.get_counts()

### Extracting the vacuum persistence probability in the physics circuit and post-selecting on states with $r=g=b=0$

In [26]:
vacAmp = []
for i in range(myruns):
    # Physics runs are the even numbered elements in the array
    temp = mit_output[2+2*i]
    physicalTotal = 0
    isKey = False
    # Post select on physical states, r=g=b=0
    for key in temp:
        if key[0]!=key[3] and key[1]!=key[4] and key[2]!=key[5]:
            physicalTotal += temp[key]
        if key == '000111':
            isKey = True
    if isKey:
        vacAmp.append(temp['000111']/physicalTotal)
    else:
        vacAmp.append(0)

In [ ]:
plt.scatter(np.arange(0, 149, 1),vacAmp)

### Extracting the vacuum persistence probability in the mitigation circuit and post-selecting on states with $r=g=b=0$

In [ ]:
mitiAmp = []
for i in range(myruns):
    # Mitigation runs are the odd numbered elements in the array
    temp = mit_output[3+2*i]
    physicalTotal = 0
    for key in temp:
        if key[0]!=key[3] and key[1]!=key[4] and key[2]!=key[5]:
            physicalTotal += temp[key]
    mitiAmp.append(temp['000111']/physicalTotal)

In [ ]:
plt.scatter(np.arange(0, 149, 1),mitiAmp)